In [15]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [16]:
# Load all CSV files

Dem_data = pd.read_excel('../Data/Data_demand.xlsx', sheet_name='Python_dem')
Dem_U = pd.read_excel('../Data/Data_demand.xlsx', sheet_name='Python_U')
Generators = pd.read_excel('../Data/Generators_Existing.xlsx', sheet_name='python')
Pmax_gen = Generators['P_max'] 
Cost_gen = Generators['cost']

Dem_data = np.array(Dem_data)
Dem_U = np.array(Dem_U)
Pmax_gen = np.array(Pmax_gen)
Cost_gen = np.array(Cost_gen)


In [17]:
Dem_data

array([[ 73.41654414,  65.68848686, 121.71690213, ..., 226.04567539,
        123.64891645,  86.94064438],
       [ 68.10274257,  60.93403283, 112.90717848, ..., 209.68476003,
        114.69935591,  80.64798463],
       [ 64.84089414,  58.01553687, 107.49937713, ..., 199.64170039,
        109.20571645,  76.78526938],
       ...,
       [140.88515567, 126.05513928, 233.57275808, ..., 433.7779793 ,
        237.28026218, 166.83768435],
       [120.16936636, 107.51995937, 199.22816001, ..., 369.99515431,
        202.39051176, 142.30582858],
       [ 99.90164103,  89.38567882, 165.62640486, ..., 307.59189475,
        168.25539542, 118.3045749 ]])

In [18]:
# PARAMETERS (we need to create another excel file with all the parameters, like the number of hours and so on) (no data on the code)

hours = 3600

N_dem = 17

N_gen_E = 16

In [19]:
# MODEL
model = gp.Model("DA market clearing")

In [20]:
# VARIABLES

d = model.addMVar((hours,N_dem), lb=0) # demand per hour in every demand
p = model.addMVar((hours,N_gen_E), lb=0) # power output per hour for every generator

In [21]:
# Constraints

max_dem = model.addConstr( d <= Dem_data, name='max_dem')



In [22]:
for h in range(hours):
    max_cap = model.addConstr( p[h,:] <= Pmax_gen, name='max_cap')

In [23]:
# OBJECTIVE FUNCTION

objective = gp.quicksum( d[h,:] @ Dem_U + p[h,:] @ Cost_gen for h in range(hours))

In [26]:
model.setObjective(objective, GRB.MAXIMIZE)
model.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 118800 rows, 118800 columns and 118800 nonzeros
Model fingerprint: 0x0eb14eec
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 118800 rows and 118800 columns
Presolve time: 0.08s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2165139e+09   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.16 seconds (0.05 work units)
Optimal objective  4.216513890e+09


In [27]:
model.objVal

4216513890.286416

In [28]:
p[5].x

array([152., 152., 350., 591.,  60., 155., 155.,   0.,   0.,   0.,   0.,
         0.,   0., 100.,   0., 200.])

In [29]:
Pmax_gen

array([152, 152, 350, 591,  60, 155, 155, 400, 400, 300, 310, 350, 500,
       100,  60, 200], dtype=int64)